In [1]:
import pandas as pd
import re

# Read the list of filenames from the configuration file
with open('file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'


# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = file
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])

In [2]:
# Group by 'stream' and aggregate unique users
users_per_stream = data.groupby("stream")["user"].unique().reset_index()

# Display the result
print(users_per_stream)

    stream                                               user
0        0  [Martin_Gales, StreamElements, Aloddin, utaBz,...
1        1  [StreamElements, Martin_Gales, Zeololz, NSAPar...
2        2  [Martin_Gales, StreamElements, sbeedy20, Banti...
3        3  [Martin_Gales, StreamElements, swedenhobohealt...
4        4  [Martin_Gales, StreamElements, jqqq777, SeeMeN...
..     ...                                                ...
67      67  [StreamElements, 1206paul_, Martin_Gales, soul...
68      68  [Martin_Gales, StreamElements, FocoExotico, Fo...
69      69  [StreamElements, Martin_Gales, Aluminiumminimu...
70      70  [Martin_Gales, StreamElements, balintboss, Alu...
71      71  [Martin_Gales, StreamElements, Aluminiumminimu...

[72 rows x 2 columns]


In [3]:
# Group by 'users' and aggregate unique streams
stream_per_users = data.groupby("user")["stream"].unique().reset_index()

# Display the result
print(stream_per_users)

              user                            stream
0        00001joel                            [4, 5]
1        00ramez00                              [14]
2         062_davi                              [37]
3     082274699958                              [55]
4         09logdog                              [53]
...            ...                               ...
5829         的显示名称                          [49, 50]
5830       豪軒99999                              [71]
5831       게이머2334                              [45]
5832           덕충2  [42, 43, 44, 45, 46, 54, 56, 57]
5833         하이드로른                          [70, 71]

[5834 rows x 2 columns]


In [4]:
stream_per_users["stream_count"] = stream_per_users["stream"].apply(len)

In [5]:
stream_per_users.head(5)

,user,stream,stream_count
0,00001joel,"[4, 5]",2
1,00ramez00,[14],1
2,062_davi,[37],1
3,082274699958,[55],1
4,09logdog,[53],1


In [6]:
stream_per_users['stream_count'].describe()

count    5834.000000
mean        3.045252
std         5.592421
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max        72.000000
Name: stream_count, dtype: float64

In [7]:
# Function to calculate the longest streak of consecutive streams
def longest_streak(streams):
    sorted_streams = sorted(streams)
    max_streak = 1
    current_streak = 1
    
    for i in range(1, len(sorted_streams)):
        if sorted_streams[i] == sorted_streams[i-1] + 1:
            current_streak += 1
        else:
            max_streak = max(max_streak, current_streak)
            current_streak = 1
    
    return max(max_streak, current_streak)

In [8]:
# Add a column with the longest streak of consecutive streams
stream_per_users["longest_streak"] = stream_per_users["stream"].apply(longest_streak)

# Display the result
stream_per_users.head(5)

,user,stream,stream_count,longest_streak
0,00001joel,"[4, 5]",2,2
1,00ramez00,[14],1,1
2,062_davi,[37],1,1
3,082274699958,[55],1,1
4,09logdog,[53],1,1


In [9]:
stream_per_users["longest_streak"].describe()

count    5834.000000
mean        1.553308
std         2.655160
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        72.000000
Name: longest_streak, dtype: float64

In [14]:
stream_per_users[stream_per_users["stream_count"] > 3]["stream_count"].describe()

count    1101.000000
mean       10.158946
std        10.077120
min         4.000000
25%         5.000000
50%         6.000000
75%        11.000000
max        72.000000
Name: stream_count, dtype: float64

In [10]:
stream_per_users[stream_per_users['user']=='kecso1g']

,user,stream,stream_count,longest_streak
3513,kecso1g,"[2, 4, 7, 10, 13, 16, 23, 31, 36, 37, 40, 44, ...",29,9


In [11]:
stream_per_users[stream_per_users['stream_count']>= 43]

,user,stream,stream_count,longest_streak
23,1206paul_,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",72,72
188,Aloddin,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14,...",66,18
191,Aluminiumminimumimmunity,"[9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21...",62,60
261,Banties1g,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",66,35
375,Cuhhsin,"[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3...",44,27
657,Grumsee,"[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14...",43,7
729,IRLKingsman,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...",56,46
759,IvanOnMyOwn,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",67,38
760,Ivana_10,"[0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15...",49,7
789,Jeddix,"[0, 1, 2, 3, 6, 7, 11, 12, 13, 15, 16, 17, 18,...",57,25


In [12]:
user_watche_these = stream_per_users[stream_per_users['user']=='balintboss']['stream']

for s in user_watche_these:
    print(s)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71]
